# Bigram Model of Neural Network

### 1.1 Load and create datasets

In [6]:
# Create list of words from names.txt
words = open('names.txt').read().splitlines()
print(len(words))

32033


In [7]:
#Designing encoder and decoder
chars = sorted(list(set(''.join(words))))

#string to integer
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0

#integer to string
itos = {i:s for s, i in stoi.items()}

In [8]:
#creating dataset
import torch
import torch.nn.functional as F

xs, ys = [], []

for w in words:
    chr_list = ['.'] + list(w) + ['.']

    for (chr, chr1) in zip(chr_list, chr_list[1:]):
        idx1 = stoi[chr]
        idx2 = stoi[chr1]
        xs.append(idx1)
        ys.append(idx2)


xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print(f"number of samples= {xs.nelement()}")


number of samples= 228146


In [9]:
#one hot encoding into 27 vector dimension space of xs

x_enc = F.one_hot(xs, num_classes=27).float()
x_enc.dtype

torch.float32

In [10]:
#network parameter
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=g, requires_grad=True)

### 1.2 Training the prepared dataset via gradient decent method of nn

In [37]:
#Training loop over 10 epochs
epochs = 10
l2 = 0.01
learning_rate = 10

for epoch in range(epochs):
    #forward pass
    x_enc = F.one_hot(xs, num_classes=27).float()
    logits = x_enc @ W # predict log-counts
    count = logits.exp() # counts, equivalent to N
    prob = count/ count.sum(1, keepdim=True) # probabilities for next character
    loss = -prob[torch.arange(num), ys].log().mean() + l2*(W**2).mean()

    #backward pass
    W.grad = None
    loss.backward()

    #optimize weights
    W.data += -learning_rate * W.grad

print(f'loss= {loss.data}')

loss= 2.4851958751678467


### 1.3 Sampling data from Trained dataset


In [39]:
#sampling of neural network
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out=[]
    idx= 0

    while True:
        x_sample = F.one_hot(torch.tensor([idx]), num_classes=27).float()
        logits = x_sample @ W
        count = logits.exp()
        prob = count/ count.sum(1, keepdim=True) 

        idx = torch.multinomial(prob, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[idx])

        if idx == 0:
            break
    
    print(''.join(out))


junide.
janasah.
p.
cfay.
a.
